In [1]:
import os
from PIL import Image
import pandas as pd

def crop_images_and_masks(input_image_dir, input_mask_dir, output_image_dir, output_mask_dir, patch_size_height, patch_size_width , output_csv_dir):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_mask_dir, exist_ok=True)
    os.makedirs(output_csv_dir, exist_ok=True)  # Create directory for CSV files if it doesn't exist

    image_files = os.listdir(input_image_dir)

    data = []  # Create a list to store image and mask information

    for image_filename in image_files:
        if image_filename.endswith('.tif'):
            image_path = os.path.join(input_image_dir, image_filename)
            mask_filename = image_filename.replace('.tif', '.tif')
            mask_path = os.path.join(input_mask_dir, mask_filename)

            if not os.path.exists(mask_path):
                print(f"Mask file not found: {mask_path}")
                continue

            # Open the image and mask using Pillow
            image = Image.open(image_path)
            mask = Image.open(mask_path)

            # Get the original dimensions
            original_width, original_height = image.size

            # Calculate the new dimensions
            new_width = original_width - (original_width % patch_size_width)
            new_height = original_height - (original_height % patch_size_height)

            # Crop the image and mask
            cropped_image = image.crop((0, 0, new_width, new_height))
            cropped_mask = mask.crop((0, 0, new_width, new_height))

            # Save or use the cropped image and mask as needed
            output_image_path = os.path.join(output_image_dir, image_filename)
            output_mask_path = os.path.join(output_mask_dir, mask_filename)

            cropped_image.save(output_image_path)
            cropped_mask.save(output_mask_path)

            # Store information in the data list
            data.append({
                'Image Filename': image_filename,
                'Original Image Width': original_width,
                'Original Image Height': original_height,
                'Cropped Image Width': new_width,
                'Cropped Image Height': new_height,
                'Mask Filename': mask_filename,
                'Original Mask Width': original_width,
                'Original Mask Height': original_height,
                'Cropped Mask Width': new_width,
                'Cropped Mask Height': new_height,
            })

    # Create a DataFrame from the data list
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    csv_filename = os.path.join(output_csv_dir, 'image_info_scaled.csv')
    df.to_csv(csv_filename, index=False)

In [ ]:
# Set the patch size and input/output directories
# patch_size = 256
patch_size_height= 256
patch_size_width = 256
input_image_directory = '../data/original_data/images/'
input_mask_directory = '../data/original_data/masks/'
output_image_directory = '../data/scaled_data/images/'
output_mask_directory = '../data/scaled_data/masks/'
output_csv_directory = '../data/csv_files/'
# Create the directories if they don't exist
os.makedirs(output_image_directory, exist_ok=True)
os.makedirs(output_mask_directory, exist_ok=True)
os.makedirs(output_csv_directory, exist_ok=True)

crop_images_and_masks(input_image_directory, input_mask_directory, output_image_directory, output_mask_directory,patch_size_height, patch_size_width,  output_csv_directory)